## 

# <center> TESOURO DIRETO com MONGDB e DASH

___

## Imports

In [28]:
# Imports
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
#pd.options.display.float_format = '{:.2f}'.format
#pd.set_option("display.max_colwidth", 150)
#pd.set_option("display.min_rows", 20)

import numpy as np
import datetime as dt
from datetime import date

# Plots
import plotly.express as px
import plotly.graph_objects as go

# dash
import dash
import dash_core_components as dcc
import dash_html_components as html
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State

In [2]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

## Funções para Busca e Tratamento de  Dados

In [3]:
def busca_titulos_tesouro_direto():
    url = 'https://www.tesourotransparente.gov.br/ckan/dataset/df56aa42-484a-4a59-8184-7676580c81e3/resource/796d2059-14e9-44e3-80c9-2d9e30b405c1/download/PrecoTaxaTesouroDireto.csv'
    df  = pd.read_csv(url, sep=';', decimal=',')
    df['Data Vencimento'] = pd.to_datetime(df['Data Vencimento'], dayfirst=True)
    df['Data Base'] = pd.to_datetime(df['Data Base'], dayfirst=True)
    multi_indice = pd.MultiIndex.from_frame(df.iloc[:, :3])
    df = df.set_index(multi_indice).iloc[: , 3:]  
    return df

In [4]:
def busca_vendas_tesouro():
    url = "https://www.tesourotransparente.gov.br/ckan/dataset/f0468ecc-ae97-4287-89c2-6d8139fb4343/resource/e5f90e3a-8f8d-4895-9c56-4bb2f7877920/download/VendasTesouroDireto.csv"
    df  = pd.read_csv(url, sep=';', decimal=',')
    df['Vencimento do Titulo'] = pd.to_datetime(df['Vencimento do Titulo'], dayfirst=True)
    df['Data Venda'] = pd.to_datetime(df['Data Venda'], dayfirst=True)
    multi_indice = pd.MultiIndex.from_frame(df.iloc[:, :3])
    df = df.set_index(multi_indice).iloc[: , 3:]  
    return df

In [5]:
def busca_recompras_tesouro():
    url = "https://www.tesourotransparente.gov.br/ckan/dataset/f30db6e4-6123-416c-b094-be8dfc823601/resource/30c2b3f5-6edd-499a-8514-062bfda0f61a/download/RecomprasTesouroDireto.csv"
    df  = pd.read_csv(url, sep=';', decimal=',')
    df['Vencimento do Titulo'] = pd.to_datetime(df['Vencimento do Titulo'], dayfirst=True)
    df['Data Resgate'] = pd.to_datetime(df['Data Resgate'], dayfirst=True)
    multi_indice = pd.MultiIndex.from_frame(df.iloc[:, :3])
    df = df.set_index(multi_indice).iloc[: , 3:]  
    return df

___

# Preços e Taxas Históricas dos Títulos Negociados

In [6]:
# Busca títulos
titulos = busca_titulos_tesouro_direto()

In [7]:
# Ordena pelo índice
titulos.sort_index(inplace=True)

# View
titulos

Taxa Compra Manha  \
Tipo Titulo                        Data Vencimento Data Base                       
Tesouro IGPM+ com Juros Semestrais 2005-07-01      2004-12-31               9.23   
                                                   2005-01-03               9.23   
                                                   2005-01-04               9.26   
                                                   2005-01-05               9.28   
                                                   2005-01-06               9.31   
...                                                                          ...   
Tesouro Selic                      2029-03-01      2023-04-12               0.18   
                                                   2023-04-13               0.18   
                                                   2023-04-14               0.19   
                                                   2023-04-17               0.19   
                                                   2023-04-18               0.19   

                                                               Taxa Venda Manha  \
Tipo Titulo                        Data Vencimento Data Base                      
Tesouro IGPM+ com Juros Semestrais 2005-07-01      2004-12-31              9.25   
                                                   2005-01-03              9.25   
                                                   2005-01-04              9.28   
                                                   2005-01-05              9.30   
                                                   2005-01-06              9.33   
...                                                                         ...   
Tesouro Selic                      2029-03-01      2023-04-12              0.19   
                                                   2023-04-13              0.19   
                                                   2023-04-14              0.20   
                                                   2023-04-17              0.20   
                                                   2023-04-18              0.20   

                                                               PU Compra Manha  \
Tipo Titulo                        Data Vencimento Data Base                     
Tesouro IGPM+ com Juros Semestrais 2005-07-01      2004-12-31          1776.67   
                                                   2005-01-03          1777.71   
                                                   2005-01-04          1778.52   
                                                   2005-01-05          1779.40   
                                                   2005-01-06          1780.21   
...                                                                        ...   
Tesouro Selic                      2029-03-01      2023-04-12         12946.40   
                                                   2023-04-13         12952.53   
                                                   2023-04-14         12958.00   
                                                   2023-04-17         12963.85   
                                                   2023-04-18         12966.59   

                                                               PU Venda Manha  \
Tipo Titulo                        Data Vencimento Data Base                    
Tesouro IGPM+ com Juros Semestrais 2005-07-01      2004-12-31         1776.51   
                                                   2005-01-03         1777.55   
                                                   2005-01-04         1778.36   
                                                   2005-01-05         1779.25   
                                                   2005-01-06         1780.06   
...                                                                       ...   
Tesouro Selic                      2029-03-01      2023-04-12        12932.12   
                                                   2023-04-13        12938.26   
             

In [8]:
# Resetar índice
titulos = titulos.reset_index()

# Nomes das colunas
titulos.columns = ['Tipo_Titulo', 'Data_Vencimento', 'Data_Base', 
                   'Taxa_Compra_Manha', 'Taxa_Venda_Manha', 
                   'PU_Compra_Manha', 'PU_Venda_Manha','PU_Base_Manha']

# Formato de data para str
titulos['Data_Vencimento'] = titulos['Data_Vencimento'].astype(str)
titulos['Data_Base'] = titulos['Data_Base'].astype(str)

___

# <center> Banco de Dados

In [9]:
# Imports
import json
import pandas as pd

import pymongo
from pymongo import MongoClient

## Inserir dados

### Cria banco, coleção e insere dados

In [11]:
# Drop collection
con = MongoClient(host="localhost", port=27017)
    
# Define pesquisa no banco de dados
db = con.TesoutoDireto
col = db.TesoutoDiretoTitulos
    
db.TesoutoDiretoTitulos.drop()

# Fecha a conexão
con.close()

In [12]:
# Cria conexão 
con = MongoClient(host="localhost", port=27017)
    
# Criar o banco de dados
db = con.TesoutoDireto
col = db.TesoutoDiretoTitulos

# Define valores
valores = []

for i in range(0, len(titulos.index)):
    valores.append(dict(titulos.iloc[i:i+1].sum()))
    
# Insere no banco
col.insert_many(valores)
        
# Fecha a conexão
con.close()

In [13]:
# Consulta tabela
con = MongoClient(host="localhost", port=27017)
    
# Define pesquisa no banco de dados
db = con.TesoutoDireto
col = db.TesoutoDiretoTitulos
    
# Listando as coleções disponíveis
print(db.list_collection_names())

# Fecha a conexão
con.close()

['TesoutoDiretoTitulos']


In [37]:
# Consulta dados
con = MongoClient(host="localhost", port=27017)
    
# Define pesquisa no banco de dados
db = con.TesoutoDireto
col = db.TesoutoDiretoTitulos
    
# Importar toda a tabela e converter para um dataframe
df_new = pd.DataFrame(list(col.find()))

# Fecha a conexão
con.close()

In [38]:
# View
df_new

,_id,Tipo_Titulo,Data_Vencimento,Data_Base,Taxa_Compra_Manha,Taxa_Venda_Manha,PU_Compra_Manha,PU_Venda_Manha,PU_Base_Manha
0,64405dc3ec0dfbd92a8dbe6a,Tesouro IGPM+ com Juros Semestrais,2005-07-01,2004-12-31,9.23,9.25,1776.67,1776.51,1827.87
1,64405dc3ec0dfbd92a8dbe6b,Tesouro IGPM+ com Juros Semestrais,2005-07-01,2005-01-03,9.23,9.25,1777.71,1777.55,1776.51
2,64405dc3ec0dfbd92a8dbe6c,Tesouro IGPM+ com Juros Semestrais,2005-07-01,2005-01-04,9.26,9.28,1778.52,1778.36,1777.32
3,64405dc3ec0dfbd92a8dbe6d,Tesouro IGPM+ com Juros Semestrais,2005-07-01,2005-01-05,9.28,9.30,1779.40,1779.25,1778.20
4,64405dc3ec0dfbd92a8dbe6e,Tesouro IGPM+ com Juros Semestrais,2005-07-01,2005-01-06,9.31,9.33,1780.21,1780.06,1779.01
...,...,...,...,...,...,...,...,...,...
129474,64405dc4ec0dfbd92a8fb82c,Tesouro Selic,2029-03-01,2023-04-12,0.18,0.19,12946.40,12932.12,12932.12
129475,64405dc4ec0dfbd92a8fb82d,Tesouro Selic,2029-03-01,2023-04-13,0.18,0.19,12952.53,12938.26,12938.26
129476,64405dc4ec0dfbd92a8fb82e,Tesouro Selic,2029-03-01,2023-04-14,0.19,0.20,12958.00,12943.72,12943.72
129477,64405dc4ec0dfbd92a8fb82f,Tesouro Selic,2029-03-01,2023-04-17,0.19,0.20,12963.85,12949.56,12949.56


___

# <center> Análise

In [39]:
# Imports
import pandas as pd
from datetime import datetime

import pymongo
from pymongo import MongoClient

___

## Dados

In [40]:
# Consulta dados
con = MongoClient(host="localhost", port=27017)
    
# Define pesquisa no banco de dados
db = con.TesoutoDireto
col = db.TesoutoDiretoTitulos
    
# Importar toda a tabela e converter para um dataframe
df = pd.DataFrame(list(col.find()))

# Fecha a conexão
con.close()

# View
df

,_id,Tipo_Titulo,Data_Vencimento,Data_Base,Taxa_Compra_Manha,Taxa_Venda_Manha,PU_Compra_Manha,PU_Venda_Manha,PU_Base_Manha
0,64405dc3ec0dfbd92a8dbe6a,Tesouro IGPM+ com Juros Semestrais,2005-07-01,2004-12-31,9.23,9.25,1776.67,1776.51,1827.87
1,64405dc3ec0dfbd92a8dbe6b,Tesouro IGPM+ com Juros Semestrais,2005-07-01,2005-01-03,9.23,9.25,1777.71,1777.55,1776.51
2,64405dc3ec0dfbd92a8dbe6c,Tesouro IGPM+ com Juros Semestrais,2005-07-01,2005-01-04,9.26,9.28,1778.52,1778.36,1777.32
3,64405dc3ec0dfbd92a8dbe6d,Tesouro IGPM+ com Juros Semestrais,2005-07-01,2005-01-05,9.28,9.30,1779.40,1779.25,1778.20
4,64405dc3ec0dfbd92a8dbe6e,Tesouro IGPM+ com Juros Semestrais,2005-07-01,2005-01-06,9.31,9.33,1780.21,1780.06,1779.01
...,...,...,...,...,...,...,...,...,...
129474,64405dc4ec0dfbd92a8fb82c,Tesouro Selic,2029-03-01,2023-04-12,0.18,0.19,12946.40,12932.12,12932.12
129475,64405dc4ec0dfbd92a8fb82d,Tesouro Selic,2029-03-01,2023-04-13,0.18,0.19,12952.53,12938.26,12938.26
129476,64405dc4ec0dfbd92a8fb82e,Tesouro Selic,2029-03-01,2023-04-14,0.19,0.20,12958.00,12943.72,12943.72
129477,64405dc4ec0dfbd92a8fb82f,Tesouro Selic,2029-03-01,2023-04-17,0.19,0.20,12963.85,12949.56,12949.56


## EDA

* Shape/Types
* Missing/Inconsistent data
* Resumo descritivo

In [41]:
# Shape
df.shape

(129479, 9)

In [42]:
# Missing data
df.isna().values.sum()

df.isna().sum()

_id                  0
Tipo_Titulo          0
Data_Vencimento      0
Data_Base            0
Taxa_Compra_Manha    0
Taxa_Venda_Manha     0
PU_Compra_Manha      0
PU_Venda_Manha       0
PU_Base_Manha        0
dtype: int64

In [43]:
# Types
#df.info()

df.dtypes

_id                   object
Tipo_Titulo           object
Data_Vencimento       object
Data_Base             object
Taxa_Compra_Manha    float64
Taxa_Venda_Manha     float64
PU_Compra_Manha      float64
PU_Venda_Manha       float64
PU_Base_Manha        float64
dtype: object

___

In [44]:
# Corrigir formatos

# Datas
df['Data_Vencimento'] = pd.to_datetime(df['Data_Vencimento'])
df['Data_Base'] = pd.to_datetime(df['Data_Base'])

# Floats
df['Taxa_Compra_Manha'] = df['Taxa_Compra_Manha'].astype(float)
df['Taxa_Venda_Manha'] = df['Taxa_Venda_Manha'].astype(float)
df['PU_Compra_Manha'] = df['PU_Compra_Manha'].astype(float)
df['PU_Venda_Manha'] = df['PU_Venda_Manha'].astype(float)
df['PU_Base_Manha'] = df['PU_Base_Manha'].astype(float)

In [45]:
# Set multi-index
multi_indice = pd.MultiIndex.from_frame(df.iloc[:, 1:3])
titulos = df.set_index(multi_indice).iloc[: , 3:]    

# View
titulos

Data_Base  \
Tipo_Titulo                        Data_Vencimento              
Tesouro IGPM+ com Juros Semestrais 2005-07-01      2004-12-31   
                                   2005-07-01      2005-01-03   
                                   2005-07-01      2005-01-04   
                                   2005-07-01      2005-01-05   
                                   2005-07-01      2005-01-06   
...                                                       ...   
Tesouro Selic                      2029-03-01      2023-04-12   
                                   2029-03-01      2023-04-13   
                                   2029-03-01      2023-04-14   
                                   2029-03-01      2023-04-17   
                                   2029-03-01      2023-04-18   

                                                    Taxa_Compra_Manha  \
Tipo_Titulo                        Data_Vencimento                      
Tesouro IGPM+ com Juros Semestrais 2005-07-01                    9.23   
                                   2005-07-01                    9.23   
                                   2005-07-01                    9.26   
                                   2005-07-01                    9.28   
                                   2005-07-01                    9.31   
...                                                               ...   
Tesouro Selic                      2029-03-01                    0.18   
                                   2029-03-01                    0.18   
                                   2029-03-01                    0.19   
                                   2029-03-01                    0.19   
                                   2029-03-01                    0.19   

                                                    Taxa_Venda_Manha  \
Tipo_Titulo                        Data_Vencimento                     
Tesouro IGPM+ com Juros Semestrais 2005-07-01                   9.25   
                                   2005-07-01                   9.25   
                                   2005-07-01                   9.28   
                                   2005-07-01                   9.30   
                                   2005-07-01                   9.33   
...                                                              ...   
Tesouro Selic                      2029-03-01                   0.19   
                                   2029-03-01                   0.19   
                                   2029-03-01                   0.20   
                                   2029-03-01                   0.20   
                                   2029-03-01                   0.20   

                                                    PU_Compra_Manha  \
Tipo_Titulo                        Data_Vencimento                    
Tesouro IGPM+ com Juros Semestrais 2005-07-01               1776.67   
                                   2005-07-01               1777.71   
                                   2005-07-01               1778.52   
                                   2005-07-01               1779.40   
                                   2005-07-01               1780.21   
...                                                             ...   
Tesouro Selic                      2029-03-01              12946.40   
                                   2029-03-01              12952.53   
                                   2029-03-01              12958.00   
                                   2029-03-01              12963.85   
                                   2029-03-01              12966.59   

                                                    PU_Venda_Manha  \
Tipo_Titulo                        Data_Vencimento                   
Tesouro IGPM+ com Juros Semestrais 2005-07-01              1776.51   
                                   2005-07-01              1777.55   
                                   2005-07-01              1778.36   
                                   2005-07-01

___

In [46]:
# Tipos de Títulos
tipos_titulos = titulos.index.droplevel(level=1).drop_duplicates().to_list()

# View
tipos_titulos

['Tesouro IGPM+ com Juros Semestrais',
 'Tesouro IPCA+',
 'Tesouro IPCA+ com Juros Semestrais',
 'Tesouro Prefixado',
 'Tesouro Prefixado com Juros Semestrais',
 'Tesouro Renda+ Aposentadoria Extra',
 'Tesouro Selic']

In [47]:
# Vencimento dos títulos
vencimentos = titulos.reset_index()[['Tipo_Titulo', 'Data_Vencimento']].astype(str)

# View
vencimentos

,Tipo_Titulo,Data_Vencimento
0,Tesouro IGPM+ com Juros Semestrais,2005-07-01
1,Tesouro IGPM+ com Juros Semestrais,2005-07-01
2,Tesouro IGPM+ com Juros Semestrais,2005-07-01
3,Tesouro IGPM+ com Juros Semestrais,2005-07-01
4,Tesouro IGPM+ com Juros Semestrais,2005-07-01
...,...,...
129474,Tesouro Selic,2029-03-01
129475,Tesouro Selic,2029-03-01
129476,Tesouro Selic,2029-03-01
129477,Tesouro Selic,2029-03-01


___

## Plot 

In [48]:
def fig(select_titulo, vencimento):
    
    
    # data
    select_titulo = 'Tesouro IPCA+'
    vencimento = '2045-05-15'
    df_titulo = titulos.loc[(select_titulo, vencimento)].reset_index()
    y = df_titulo['Taxa_Compra_Manha']
    
    # range y_axis
    rangee = []
    if min(y) < 1:
        rangee.append(min(y)-0.01)
        rangee.append(max(y)+0.01)
    else:
        rangee.append(min(y)*0.995)
        rangee.append(max(y)*1.05)

    
    # plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        name = '',
        x = df_titulo['Data_Base'],
        y = y,
        mode = 'lines', 
        fillcolor = ('rgba(46,139,87,0.05)'),
        stackgroup = 'one',
        opacity = None,
        line = dict(width = 2, color = 'SeaGreen')

    ))

    # layout
    fig.update_layout(title = select_titulo, title_x = 0.5, template = 'plotly_white')

    fig.update_xaxes(showspikes = True, spikemode = 'across', spikesnap = 'cursor', spikedash = 'dot', spikethickness = 0.5)
    fig.update_yaxes(showspikes = False, spikemode = 'across', spikesnap = 'cursor', spikedash = 'dot', spikethickness = 0.15)

    fig.update_xaxes(title = '', showgrid = False, gridwidth = None, gridcolor = None,
                     showline = True, linecolor = 'rgba(105,105,105,0.7)', linewidth = 1.4)                
    fig.update_yaxes(title = '', showgrid = True, gridwidth = None, gridcolor = 'rgba(220,220,220,0.2)',
                     showline = True, linecolor = 'white', linewidth = 1, 
                     range = rangee)

    #buttons
    fig.update_xaxes(rangeslider_visible = False,
                     rangeselector = dict(
                         buttons = list([
                             dict(count=1, label="1m", step="month", stepmode="backward"),
                             dict(count=6, label="3m", step="month", stepmode="backward"),
                             dict(count=1, label="1y", step="year", stepmode="backward"),
                             dict(step="all")
                             ]) )  )
    
    #fig.update_layout( autosize = True, width = None, height = 550)

    return fig

In [49]:
select_titulo = 'Tesouro IPCA+'
vcto_porTitulo = vencimentos.loc[vencimentos['Tipo_Titulo'] == select_titulo]

vcto_porTitulo

,Tipo_Titulo,Data_Vencimento
14941,Tesouro IPCA+,2015-05-15
14942,Tesouro IPCA+,2015-05-15
14943,Tesouro IPCA+,2015-05-15
14944,Tesouro IPCA+,2015-05-15
14945,Tesouro IPCA+,2015-05-15
...,...,...
29075,Tesouro IPCA+,2045-05-15
29076,Tesouro IPCA+,2045-05-15
29077,Tesouro IPCA+,2045-05-15
29078,Tesouro IPCA+,2045-05-15


In [50]:
vencimento = '2045-05-15'
titulo = titulos.loc[(select_titulo, vencimento)]
y = titulo['PU_Compra_Manha']

fig(titulo, vencimento)

___

# Dash

In [51]:
# initialise the app 
app = dash.Dash(__name__, external_stylesheets = [dbc.themes.BOOTSTRAP])

# styling the sidebar
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "18rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
}

# padding for the page content
CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

sidebar = html.Div(
    [
        html.H2('Filtros', className = 'display-4'),
        html.Hr(),
        html.P(
            'Título', className = 'lead'            
        ),
        
        dcc.Dropdown(id = 'titulo', multi = False, value = 'Tesouro IPCA+',
                    options = [{'label': x, 'value': x} for x in sorted(titulos.index.droplevel(level=1).drop_duplicates().to_list())]),
        html.P(),
        html.P(
            'Vencimento', className = 'lead'
        ),
        
        dcc.Dropdown(id = 'vcto', multi = False, value = '2045-05-15', 
                     options = []),
        html.P(),
        html.P(
            'Parâmetros', className = 'lead'
        ),
        
        dbc.RadioItems(
            id = "precoTaxa",
            className = "btn-group",
            inputClassName = "btn-check",
            labelClassName = "btn", #btn-outline-primary
            labelCheckedClassName = "active",
            options = [
                {"label": "Preço", "value": 1},
                {"label": " Taxa", "value": 2},
            ],
            value = 1,
        ),
        html.Hr()
        
    ],
    style = SIDEBAR_STYLE,
)

content = html.Div(id = 'page_content', 
                   children = [
                       dbc.Row([
                           dbc.Col(
                               html.H1('Tesouro Direto', className = 'text-center display-3'),
                               width = 12
                           )
                       ]),
                       html.Hr(),
                       
                        dbc.Row([
                           dbc.Col([
                                dcc.Graph(id = 'graph', figure = {})
                           ], xs = 12, sm = 12, md = 12, lg = 12, xl = 12),
                            
                       ], justify = 'around'),
                   ], 
                   style = CONTENT_STYLE)


# Define de app
app.layout = html.Div([
    sidebar,
    content
])


# callback
# Populate the counties dropdown with options and values
@app.callback(
    Output(component_id = 'vcto', component_property = 'options'),
    Input(component_id = 'titulo', component_property = 'value'),
    prevent_initial_call = False )

def setVctos(chosenTitle):
    vcto_porTitulo = vencimentos.reset_index().loc[vencimentos.reset_index()['Tipo_Titulo'] == chosenTitle]
    return [{'label': x, 'value': x} for x in sorted(vcto_porTitulo['Data_Vencimento'].unique())]


@app.callback(
    Output('graph', 'figure'),
    Input('titulo', 'value'),
    Input('vcto', 'value'),
    Input('precoTaxa', 'value'),
    prevent_initial_call = False)    

def update_graph(chosenTitle, dates, precoTaxa):
    
    if len(chosenTitle) == 0:
        return no_update
    
    else:
        # data
        df_titulos = titulos.reset_index()
        df_titulos = df_titulos[(df_titulos['Tipo_Titulo'] == chosenTitle) & (df_titulos['Data_Vencimento'] == dates)]
        if precoTaxa == 1:
            y = df_titulos['PU_Compra_Manha']
        else:
            y = df_titulos['Taxa_Compra_Manha']
        
        # range y_axis
        rangee = []
        if min(y) < 1:
            rangee.append(min(y)-0.01)
            rangee.append(max(y)+0.01)
    
        else:
            rangee.append(min(y)*0.995)
            rangee.append(max(y)*1.05)

    
        # plot
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            name = '',
            x = df_titulos['Data_Base'],
            y = y,
            mode = 'lines', 
            fillcolor = ('rgba(46,139,87,0.05)'),
            stackgroup = 'one',
            opacity = None,
            line = dict(width = 2, color = 'SeaGreen')

        ))

        # layout
        fig.update_layout(title = chosenTitle, title_x = 0.5, template = 'plotly_white')

        fig.update_xaxes(showspikes = True, spikemode = 'across', spikesnap = 'cursor', spikedash = 'dot', spikethickness = 0.5)
        fig.update_yaxes(showspikes = False, spikemode = 'across', spikesnap = 'cursor', spikedash = 'dot', spikethickness = 0.15)

        fig.update_xaxes(title = '', showgrid = False, gridwidth = None, gridcolor = None,
                         showline = True, linecolor = 'rgba(105,105,105,0.7)', linewidth = 1.4)                
        fig.update_yaxes(title = '', showgrid = True, gridwidth = None, gridcolor = 'rgba(220,220,220,0.2)',
                         showline = True, linecolor = 'white', linewidth = 1, 
                         range = rangee)

        #buttons
        fig.update_xaxes(rangeslider_visible = False,
                         rangeselector = dict(
                             buttons = list([
                                 dict(count=1, label="1m", step="month", stepmode="backward"),
                                 dict(count=6, label="3m", step="month", stepmode="backward"),
                                 dict(count=1, label="1y", step="year", stepmode="backward"),
                                 dict(step="all")
                                 ]) )  )   
        
        fig.update_layout(autosize = True, width = None, height = 600)
        
        
        return fig
    

# Run the app
if __name__ == '__main__':
    app.run_server(debug = False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050 (Press CTRL+C to quit)
127.0.0.1 - - [19/Apr/2023 18:32:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 18:32:44] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 18:32:44] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 18:32:44] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Apr/2023 18:32:44] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Apr/2023 18:32:44] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [19/Apr/2023 18:32:44] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 18:32:44] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\fernando.sartor_upni\anaconda3\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\fernando.sartor_upni\anaconda3\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\fernando.sartor_upni\anaconda3\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\fernando.sartor_upni\anaconda3\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\fernando.sartor_upni\anaconda3\lib\site-packages\dash\dash.py", line 1468, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "C:\Users\fernando.sartor_upni\anaconda3\lib\site-packages\dash\_callback.py", line 151, in add_conte

127.0.0.1 - - [19/Apr/2023 18:32:44] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\fernando.sartor_upni\anaconda3\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\fernando.sartor_upni\anaconda3\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\fernando.sartor_upni\anaconda3\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\fernando.sartor_upni\anaconda3\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\fernando.sartor_upni\anaconda3\lib\site-packages\dash\dash.py", line 1468, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "C:\Users\fernando.sartor_upni\anaconda3\lib\site-packages\dash\_callback.py", line 151, in add_conte

...

127.0.0.1 - - [19/Apr/2023 18:33:17] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\fernando.sartor_upni\anaconda3\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\fernando.sartor_upni\anaconda3\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\fernando.sartor_upni\anaconda3\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\fernando.sartor_upni\anaconda3\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\fernando.sartor_upni\anaconda3\lib\site-packages\dash\dash.py", line 1468, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "C:\Users\fernando.sartor_upni\anaconda3\lib\site-packages\dash\_callback.py", line 151, in add_conte

127.0.0.1 - - [19/Apr/2023 18:33:17] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\fernando.sartor_upni\anaconda3\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\fernando.sartor_upni\anaconda3\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\fernando.sartor_upni\anaconda3\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\fernando.sartor_upni\anaconda3\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\Users\fernando.sartor_upni\anaconda3\lib\site-packages\dash\dash.py", line 1468, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File "C:\Users\fernando.sartor_upni\anaconda3\lib\site-packages\dash\_callback.py", line 151, in add_conte

127.0.0.1 - - [19/Apr/2023 18:33:20] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [19/Apr/2023 18:33:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 18:33:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 18:33:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 18:33:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 18:33:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 18:33:42] "POST /_dash-update-component HTTP/1.1" 200 -


___